In [ ]:
import pandas as pd
from datetime import time
df = pd.read_csv("F:\Data Analyst\Tula Capital_Likedin Company\market_data.csv",parse_dates=[['currentdate','currenttime']])
df = df.rename(columns={'Unnamed: 0':'S.No','currentdate_currenttime':'date-time'})
df['date'] = df['date-time'].dt.date
df['time'] = df['date-time'].dt.time
df = df[['S.No','date-time','date','time','high','low','close','symbol']]
# print(df.head())
# print(df.info())

# defining parameters
target = 0.005
stop_loss = 0.0025
exit_time = time(15,15)

# creating an empty list to store the dictonaries and log the data to csv
trades = []

# collecting unique symbols
symbols = df['symbol'].unique()
for symbol in symbols:
    # we are filtering the data for every symbol and string it into 
    # new data frame i.e symbol_data
    # for every iteration of the for loop we have a dataframe of particular symbol only
    symbol_data = df[df['symbol']==symbol].copy()
    #print(symbol_data)
    
    # collecting unique dates
    unique_dates = symbol_data['date'].unique()
    # running the for loop based on each unique dates for the particular symbol
    for date in unique_dates:
        data_date = symbol_data[symbol_data['date']==date].copy()
        # sorting the values of particular symbol on particular date
        data_date= data_date.sort_values(by='time')
        #print(date)
        
        if data_date.empty:
            continue
        
        first_candle = data_date.iloc[0]
        entry_high = first_candle['high']
        entry_low = first_candle['low']
        entry_close = first_candle['close']
        
        #print(entry_high,entry_low,entry_close)
        
        # intaiting the boolean expressions with False making them like not yet enterd market
        long_entered = False
        short_entered = False
        long_exit = False
        short_exit = False
        long_entry_price = None
        long_curr_time = None
        short_entry_price = None
        short_curr_time = None
        
        for i,row in data_date.iterrows():
            curr_time = row['time']
            curr_price = row['close']
            curr_dt = row['date-time']
            # print(curr_dt,curr_price,curr_time)
            
             # time exit logic
            if curr_time >= exit_time:
                    # exiting long trade
                    if long_entered and not long_exit:
                        trades.append(
                        {
                            'date': row['date'],
                            'symbol' : row['symbol'],
                            'entry_type' : 'Long',
                            'entry_time' : long_entry_time,
                            'exit_time' : curr_time,
                            'entry_price' : long_entry_price,
                            'exit_price' : curr_price,
                            'pnl'   : round((curr_price-long_entry_price),2),
                            'exit_reason' : 'Time Exit'
                        }
                        )
                        long_exit = True
                    
                    # exiting short trade
                    if short_entered and not short_exit:
                        trades.append(
                        {
                            'date': row['date'],
                            'symbol' : row['symbol'],
                            'entry_type' : 'Short',
                            'entry_time' : short_entry_time,
                            'exit_time' : curr_time,
                            'entry_price' : short_entry_price,
                            'exit_price' : curr_price,
                            'pnl'   : round((short_entry_price-curr_price),2),
                            'exit_reason' : 'Time Exit'
                        }
                        )
                        short_exit = True
                    break
                
            # long entry
            if not long_entered and row['high'] > entry_high:
                # as we dont know the particular second when did the breakout happen so we assume entry_high as the 
                # long_entry_price. our assumption is we have buy the stock as soon as it went break out
                long_entry_price = entry_high
                long_entry_time = curr_time
                target1 = round(long_entry_price*(1+target),2)
                stop = round(long_entry_price*(1-stop_loss),2)
                long_entered = True
            
            # exiting the long trade due to target condition
            if long_entered and not long_exit:
                if row['high'] >= target1:
                    trades.append(
                        {
                            'date': row['date'],
                            'symbol' : row['symbol'],
                            'entry_type' : 'Long',
                            'entry_time' : long_entry_time,
                            'exit_time' : curr_time,
                            'entry_price' : long_entry_price,
                            'exit_price' : target1,
                            'pnl'   : round((target1 - long_entry_price),2),
                            'exit_reason' : 'Target'
                        }
                    )
                    long_exit = True
                
                # exiting the long trade due to stop condition
                elif row['low'] <= stop:
                    trades.append(
                        {
                            'date': row['date'],
                            'symbol' : row['symbol'],
                            'entry_type' : 'Long',
                            'entry_time' : long_entry_time,
                            'exit_time' : curr_time,
                            'entry_price' : long_entry_price,
                            'exit_price' : stop,
                            'pnl'   : round((stop - long_entry_price),2),
                            'exit_reason' : 'Stop Loss'
                        }
                    )
                    long_exit = True
                    
            # short entry
            if not short_entered and row['low'] < entry_low:
                # as we dont know the particular second when did the breakout happen so we assume entry_low as the 
                # short_entry_price. our assumption is we have sell the stock as soon as it went break out
                short_entry_price = entry_low
                short_entry_time = curr_time
                target2 = round(short_entry_price*(1-target),2)
                stop = round(short_entry_price*(1+stop_loss),2)
                short_entered = True
            
            # exiting the short trade due to target condition
            if short_entered and not short_exit:
                if row['low'] <= target2:
                    trades.append(
                        {
                            'date': row['date'],
                            'symbol' : row['symbol'],
                            'entry_type' : 'Short',
                            'entry_time' : short_entry_time,
                            'exit_time' : curr_time,
                            'entry_price' : short_entry_price,
                            'exit_price' : target2,
                            'pnl'   : round((short_entry_price-target2),2),
                            'exit_reason' : 'Target'
                        }
                    )
                    short_exit = True
                
                # exiting the short trade due to stop condition
                elif row['high'] >= stop:
                    trades.append(
                        {
                            'date': row['date'],
                            'symbol' : row['symbol'],
                            'entry_type' : 'Short',
                            'entry_time' : short_entry_time,
                            'exit_time' : curr_time,
                            'entry_price' : short_entry_price,
                            'exit_price' : stop,
                            'pnl'   : round((short_entry_price-stop),2),
                            'exit_reason' : 'Stop Loss'
                        }
                    )
                    short_exit = True
                    
   
                
# creating a datafram from using the trades list where all the logs are stored
trade_log = pd.DataFrame(trades)

# saving these logs to csv file
trade_log.to_csv('F:\Data Analyst\Tula Capital_Likedin Company/trade_logs.csv',index=False)

print('trade_logs csv file has been succesfully created')

        

trade_logs csv file has been succesfully created
